# **Лабораторна робота No2**

**Наука про дані:** підготовчий етап

**Мета роботи:** ознайомитися з основними кроками по роботі з даними –
workflow від постановки задачі до написання пояснювальної записки,
зрозуміти постановку задачі та природу даних, над якими виконується
аналітичні операції
Основні поняття: сирі дані (raw data), підготовка даних (data preparation)

### **Хід виконання роботи**

### Завдання 1
1. Створити env в якому будуть встановлені всі необхідні бібліотеки та
налаштування для данної лабораторної роботи


Імпортуємо модулі

In [4]:
import urllib.request as urlib2
import pandas as pd
import numpy as np
import datetime
import os
import glob

### Завдання 2
2. Для кожної із адміністративних одиниць України завантажити тестові
структуровані файли, що містять значення VHI-індексу. Ця процедура
має бути автоматизована, параметром процедури має бути індекс
(номер) області. При зберіганні файлу до його імені потрібно додати
дату та час завантаження;

Створюємо папку або перевіряємо її наявність

In [5]:
directory_name = "node"
try: 
    os.mkdir(directory_name)
    print("Directory: ", directory_name, "created successfully")
except PermissionError:
    print("Directory: ", directory_name, "already created")
except Exception as e:
    print("Error: ", e)

Error:  [Errno 17] File exists: 'node'


Зчитуємо дату та час

In [6]:
now = datetime.datetime.now()
current_time = now.strftime("data(%Y-%m-%d) and time(%H:%M:%S)")
print(current_time)

data(2024-12-16) and time(15:44:50)


Ствоюємо словник

In [7]:
dict = {
    1 : 24,
    2 : 25,
    3 : 5,
    4 : 6,
    5 : 27,
    6 : 23,
    7 : 26,
    8 : 7,
    9 : 11,
    10 : 13,
    11 : 14,
    12 : 15,
    13 : 16,
    14 : 17,
    15 : 18,
    16 : 19,
    17 : 21,
    18 : 22,
    19 : 8,
    20 : 9,
    21 : 10,
    22 : 1,
    23 : 3,
    24 : 2,
    25 : 4,
    26 : 12,
    27 : 20
    }

Завантажуємо файли в папку

In [8]:
file_path = "node"
for i in range(1,29):
    name = 'NOAA_ID'+str(i)
    if glob.glob(os.path.join(directory_name, name)):
        print("Файл існує:", str(i))
        continue
    url = "https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID={}&year1=1981&year2=2024&type=Mean".format(i)
    wp = urlib2.urlopen(url)
    text = wp.read()
    out = open(os.path.join(file_path, name+'_'+current_time+'.csv'),'wb')
    out.write(text)
    out.close()


### Завдання 3
3. Зчитати завантажені текстові файли у фрейм (детальніше про роботу
із фреймами буде розказано у подальших лабораторних роботах).
Імена стовбців фрейму мають бути змістовними та легкими для
сприйняття (не повинно бути спеціалізованих символів, пробілів
тощо). Ця задача має бути реалізована у вигляді окремої процедури,
яка на вхід приймає шлях до директорії, в якій зберігаються файли;

Створюємо головний Датафрейм

In [9]:
headers = ['Year', 'Week', 'SMN', 'SMT', 'VCI', 'TCI', 'VHI', 'empty']
data = pd.DataFrame(columns=headers)
data

,Year,Week,SMN,SMT,VCI,TCI,VHI,empty


Обʼєднуємо файли в один та очищуємо

In [10]:
for i in range(1,29):
    name = f"NOAA_ID{i}_*.csv"
    file_list = glob.glob(os.path.join(directory_name, name))
    for file_path2 in file_list:
        df = pd.read_csv(file_path2, header = 1, names = headers)
        df = df.drop(df.loc[df['VHI'] == -1].index)
        df['area'] = i
        df = df[~df['Year'].astype(str).str.contains('<|>', regex=True)]
        #df = df.dropna()
        data = pd.concat([data, df], ignore_index=True)
data = data.drop(data.columns[[-2]], axis=1)
data
    


/var/folders/d7/mv12n9yn7c1gy3kr948lcfth0000gn/T/ipykernel_24192/2587880211.py:10: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  data = pd.concat([data, df], ignore_index=True)


,Year,Week,SMN,SMT,VCI,TCI,VHI,area
0,1982,2.0,0.054,262.29,46.83,31.75,39.29,1.0
1,1982,3.0,0.055,263.82,48.13,27.24,37.68,1.0
2,1982,4.0,0.053,265.33,46.09,23.91,35.00,1.0
3,1982,5.0,0.050,265.66,41.46,26.65,34.06,1.0
4,1982,6.0,0.048,266.55,36.56,29.46,33.01,1.0
...,...,...,...,...,...,...,...,...
122103,2024,45.0,0.036,310.59,16.57,48.36,32.47,28.0
122104,2024,46.0,0.036,309.78,16.68,37.94,27.31,28.0
122105,2024,47.0,0.037,308.59,18.82,33.32,26.07,28.0
122106,2024,48.0,0.038,307.15,20.64,31.09,25.87,28.0


### Завдання 4
4. Реалізувати процедуру, яка змінить індекси областей, які використані
на порталі NOAA на наведені

In [11]:
for i in range(1,28):
    data.loc[data["area"]==dict[i], "area"] = i

Вивід результату

In [12]:
data

,Year,Week,SMN,SMT,VCI,TCI,VHI,area
0,1982,2.0,0.054,262.29,46.83,31.75,39.29,22.0
1,1982,3.0,0.055,263.82,48.13,27.24,37.68,22.0
2,1982,4.0,0.053,265.33,46.09,23.91,35.00,22.0
3,1982,5.0,0.050,265.66,41.46,26.65,34.06,22.0
4,1982,6.0,0.048,266.55,36.56,29.46,33.01,22.0
...,...,...,...,...,...,...,...,...
122103,2024,45.0,0.036,310.59,16.57,48.36,32.47,28.0
122104,2024,46.0,0.036,309.78,16.68,37.94,27.31,28.0
122105,2024,47.0,0.037,308.59,18.82,33.32,26.07,28.0
122106,2024,48.0,0.038,307.15,20.64,31.09,25.87,28.0


### Завдання 5
5. Реалізувати процедури для формування вибірок наступного виду
(включаючи елементи аналізу):
- Ряд VHI для області за рік, пошук екстремумів (min та max);
- Ряд VHI за всі роки для області, виявити роки з екстремальними
посухами, які торкнулися більше вказаного відсотка області;
- Аналогічно для помірних посух

Вивід min/max

In [13]:
def min_max(year, area):
    print("MIN: ", data["VHI"][(data["Year"] == f"{year}") & (data["area"] == area)].min())
    print("MAX: ", data["VHI"][(data["Year"] == f"{year}") & (data["area"] == area)].max())
min_max(1991, 5)

MIN:  30.93
MAX:  59.74


Вивід екстремальних посух

In [14]:
def extremum(area):
    print(data.loc[(data["VHI"] < 15) & (data["area"] == area), ["VHI", "area"]])
extremum(22)

          VHI  area
948     14.64  22.0
949     11.82  22.0
950     10.81  22.0
951     10.68  22.0
952     12.30  22.0
953     14.24  22.0
3127    14.64  22.0
3128    11.82  22.0
3129    10.81  22.0
3130    10.68  22.0
3131    12.30  22.0
3132    14.24  22.0
101252  12.26  22.0
101253  11.28  22.0
101254  11.25  22.0
101255  11.38  22.0
101256  12.91  22.0
101257  14.20  22.0
103431  12.26  22.0
103432  11.28  22.0
103433  11.25  22.0
103434  11.38  22.0
103435  12.91  22.0
103436  14.20  22.0


Вивід помірних посух

In [15]:
def pomir(area):
    print(data.loc[(data["VHI"]<35) & (data["VHI"]>=15) &(data["area"] == area), ["VHI", "area"]])
pomir(22)

          VHI  area
3       34.06  22.0
4       33.01  22.0
5       31.65  22.0
6       31.40  22.0
7       31.73  22.0
...       ...   ...
104650  28.83  22.0
104651  29.73  22.0
104652  30.45  22.0
104653  31.87  22.0
104654  34.63  22.0

[1083 rows x 2 columns]
